In [1]:
import pickle
import numpy as np
from gather import gatherFromUrl

with open('cutData.ndas','rb') as f:
    gamelist,userlist,R,I,It = pickle.load(f)

_GL, _UL = I.shape
r=np.zeros((_GL,1),dtype="float32")
i=np.zeros((_GL,1),dtype="bool")
zero=np.zeros_like(r)

user=input("Type username : ")
urlhead="https://www.boardgamegeek.com/xmlapi2/collection?username="
url=urlhead+user+"&subtype=boardgame&stats=1&rated=1"
infos=gatherFromUrl(url,"item",notyet=1)
for item in infos:
    game=item.get("objectid")
    if game in gamelist:
        gid=gamelist.index(game)
        rating=float(item.find("rating").get("value"))
        r[gid]=rating
        i[gid]=True
        
_IL=np.sum(i)
print(_IL)    

Type username : GregSLee
21


In [2]:
print("Finding base")

import tensorflow as tf

with open('baseline.ndas', 'rb') as f:
    bm,bg,bu = pickle.load(f)

b = tf.Variable( tf.random.normal((1,)) )
bias=bm+np.expand_dims(bg,1)+b

cost = tf.math.reduce_mean(b**2)
print("...")
sqd = tf.squared_difference(r,bias)
print("...")
loss = tf.math.reduce_sum( tf.where(i, sqd, zero) )/_IL
print("...")
train = tf.train.AdamOptimizer(learning_rate=0.1).minimize(loss+0.0001*cost)
print("...")

init = tf.global_variables_initializer()
sess=tf.Session()
sess.run(init)
print("initiated")
for _ in range(64):
    sess.run(train)
    print(sess.run(loss),sess.run(cost))

bias=sess.run(bias)

Finding base
Instructions for updating:
Colocations handled automatically by placer.
...
...
...
...
initiated
2.7268496 0.2957454
2.7257376 0.20859487
2.7321172 0.16466686
2.7337713 0.15749554
2.7304914 0.17269094
2.7263694 0.20161867
2.7243948 0.23781994
2.725208 0.27360743
2.727284 0.30030227
2.7285266 0.3118879
2.7280562 0.3077099
2.72646 0.29133776
2.7249281 0.26812544
2.7243562 0.24357817
2.7248516 0.2224335
2.7257855 0.20801255
2.7263434 0.2018797
2.7261388 0.20400421
2.72539 0.21319975
2.724645 0.22743098
2.724356 0.24393536
2.7245955 0.25944933
2.7250586 0.27080813
2.725335 0.27581125
2.7252226 0.2738667
2.7248354 0.26603758
2.72447 0.2545687
2.7243593 0.24220583
2.7245173 0.23154464
2.724758 0.22453089
2.7248635 0.22218281
2.7247574 0.2245416
2.7245386 0.23077376
2.7243776 0.23934819
2.7243755 0.2482937
2.724492 0.25558722
2.7246015 0.2596424
2.7246044 0.259734
2.724506 0.25616387
2.7243948 0.25010532
2.7243567 0.24321759
2.7244031 0.23718722
2.724474 0.23333095
2.724497 0.23

In [3]:
print("Weight a second (:")

with open('gameinfo.lists','rb') as f:
    namelist, minplist, maxplist, weightlist, catmeclist = pickle.load(f)

Iu=np.sum(i,axis=0,keepdims=True)    
w = np.expand_dims( np.array(weightlist, dtype="float32"), 1)

cw = tf.Variable( tf.random.normal((1,), mean=2.0) )
bw = tf.Variable( tf.random.normal((1,), mean=0.1, stddev=0.01), constraint=lambda x: tf.where(x>0.01,x,0.01*tf.ones_like(x)) )
wd = tf.abs(w-cw)
wsum = tf.math.reduce_sum(tf.where(i,wd,zero), axis=0, keepdims=True)
rp = bias+bw*(wsum/Iu-wd)

sqd = tf.squared_difference(r,rp)
print("...")
loss = tf.math.reduce_sum( tf.where(i, sqd, zero) )/_IL
print("...")
train = tf.train.AdamOptimizer(learning_rate=0.1).minimize(loss)

print("...")

init = tf.global_variables_initializer()
sess=tf.Session()
sess.run(init)
print("initiated")
for _ in range(64):
    sess.run(train)
    print(sess.run(loss))

rp=sess.run(rp)

Weight a second (:
...
...
Instructions for updating:
Use tf.cast instead.
...
initiated
2.5061202
2.404712
2.315058
2.2431753
2.1835027
2.1358225
2.099653
2.0742302
2.058489
2.0510702
2.0503588
2.0545683
2.0618606
2.07049
2.0789435
2.0860493
2.091031
2.0935104
2.0934627
2.0911436
2.0870042
2.0816066
2.075546
2.0693939
2.0636444
2.0586846
2.0547726
2.0520337
2.0504642
2.0499508
2.0502949
2.051243
2.0525231
2.0538745
2.055075
2.0559602
2.0564332
2.056465
2.0560873
2.0553784
2.0544472
2.0534146
2.0523968
2.0514927
2.0507734
2.0502775
2.0500112
2.049953
2.0500584
2.0502706
2.0505302
2.0507815
2.05098
2.0510976
2.051123
2.0510588
2.0509229
2.0507388
2.0505354
2.0503392
2.0501723
2.0500488
2.0499754
2.04995


In [4]:
print("Finding matrix")

with open('MF.ndas', 'rb') as f:
    Fg,Fu = pickle.load(f)
_N=Fg.shape[1]
f = tf.Variable( tf.random.normal((_N,1)) )
    
rp2 = Fg@f + bias

cost = tf.math.reduce_mean(f**2)
print("...")
sqd = tf.squared_difference(r,rp2)
print("...")
loss = tf.math.reduce_sum( tf.where(i, sqd, zero) )/_IL
print("...")
train = tf.train.AdamOptimizer(learning_rate=0.1).minimize(loss+0.5*cost)

print("...")

init = tf.global_variables_initializer()
sess=tf.Session()
sess.run(init)
print("initiated")
for _ in range(32):
    sess.run(train)
    print(sess.run(loss))

rp2=sess.run(rp2)

Finding matrix
...
...
...
...
initiated
7.0779676
6.212813
5.4533873
4.7983365
4.249151
3.804267
3.4559393
3.1925857
3.0013673
2.868651
2.7799864
2.720883
2.6787446
2.6442664
2.6110847
2.574881
2.5329893
2.4844859
2.4300704
2.371417
2.3104253
2.2489307
2.1888504
2.132182
2.0806317
2.0352545
1.9963982
1.9638807
1.9370956
1.9149998
1.8962302
1.8794433


In [5]:
print("Blending")

with open('blend.nda', 'rb') as f:
    ratios = pickle.load(f)

rtrivial = np.sum(I*R,axis=1,keepdims=True)/np.sum(I,axis=1,keepdims=True)
rps=np.stack((rtrivial,bias,rp,rp2))
x=rps*ratios[:,tf.newaxis,tf.newaxis]
rb=np.sum(x,axis=0)

np.sum( i*(r-rb)**2 )/_IL    

Blending


1.8625333926417706

In [6]:
z=zip( list(rb[i[:,0]]), list(r[i[:,0]]), list(np.array(namelist)[i[:,0]]))
z=sorted(list(z),reverse=True)
for qqq in z:
    print(qqq)


(array([6.96064977]), array([8.], dtype=float32), 'Gizmos')
(array([6.94959285]), array([6.], dtype=float32), 'Dixit')
(array([6.91980144]), array([6.], dtype=float32), 'Splendor')
(array([6.84250464]), array([9.], dtype=float32), 'Las Vegas')
(array([6.65156077]), array([7.], dtype=float32), '7 Wonders Duel')
(array([6.50825621]), array([7.], dtype=float32), 'Lost Cities')
(array([6.41868086]), array([5.], dtype=float32), 'Blokus')
(array([6.37010099]), array([5.], dtype=float32), 'Carcassonne: South Seas')
(array([6.3648345]), array([7.], dtype=float32), 'BANG!')
(array([6.27874503]), array([7.], dtype=float32), 'Ricochet Robots')
(array([6.26968046]), array([7.], dtype=float32), 'Rummikub')
(array([6.13783107]), array([8.], dtype=float32), 'Citadels')
(array([5.96315902]), array([2.], dtype=float32), 'YINSH')
(array([5.95819815]), array([6.], dtype=float32), 'Cockroach Poker')
(array([5.88809279]), array([7.], dtype=float32), 'Halli Galli')
(array([5.84318627]), array([6.], dtype=fl

In [7]:
rb=list(rb[:,0])
z=zip(rb,namelist)
z=sorted(list(z),reverse=True)
for rating,game in z[:50]:
    print(rating,game)

8.365807297564071 Pandemic Legacy: Season 1
7.990313532063712 Aeon's End: Legacy
7.840424220586513 Pandemic Legacy: Season 2
7.7674557113073455 Aeon's End: War Eternal
7.747262063698918 Cards Against Humanity
7.6764066574699745 Aeon's End: The New Age
7.604096415359093 Legacy of Dragonholt
7.586199162643515 Gloomhaven
7.5683737754228675 Monikers
7.557079419920456 Star Realms: Colony Wars
7.55074518923323 Time's Up! Academy
7.542548980012514 Codenames: Duet
7.538825738386384 Nemesis
7.485067530537596 Watergate
7.464806476477249 Time's Up! Deluxe
7.461932390186728 Telestrations: 12 Player Party Pack
7.442108920805854 Just One
7.431381721422531 Baseball Highlights: 2045 – Deluxe Edition
7.416933218655085 Eat Poop You Cat
7.4120081350007725 Mythos Tales
7.393415228277918 Mega Civilization
7.386511391981603 Concordia Venus
7.361268669159909 Too Many Bones: Undertow
7.345705971968922 1817
7.343802999207547 Codenames
7.343463956670886 Star Realms: Frontiers
7.342717201230622 Wingspan
7.340707